In [1]:
import pandas as pd
from datetime import datetime
import plotly.express as px

In [2]:
# https://www.data.go.kr/data/15049591/fileData.do <= 표준산업분류코드 데이터
def std_convert(str):
    return f"0{str}" if len(str) == 1 else str
STD_TOBZ_M_df = pd.read_csv("dataset/고용노동부_표준산업분류코드.csv")
STD_TOBZ_M_df['고용업종코드(중분류)'] = STD_TOBZ_M_df['고용업종코드(중분류)'].astype(int).astype(str)
STD_TOBZ_M_df['고용업종코드(중분류)'] = STD_TOBZ_M_df['고용업종코드(중분류)'].apply(lambda x : std_convert(x))
STD_TOBZ_M_df['고용업종코드(소분류)'] = STD_TOBZ_M_df['고용업종코드(소분류)'].astype(str)

STD_TOBZ_M_dic = dict(zip(STD_TOBZ_M_df['고용업종코드(중분류)'], STD_TOBZ_M_df['고용업종명(중분류)'])) # 표준산업분류 - 중분류 dict
STD_TOBZ_S_dic = dict(zip(STD_TOBZ_M_df['고용업종코드(소분류)'], STD_TOBZ_M_df['고용업종명(소분류)'])) # 표준산업분류 - 소분류 dict

In [6]:
# km data preprocessing
def exc_age(age):
    age = age.split(".")[-1].split("대")[0]
    try:
        return "60+" if int(age) >= 60 else age
    except:
        return age
km_data = pd.read_csv("dataset/국민카드.csv", encoding='cp949')
km_data = km_data[~km_data['표준산업세세분류코드'].isna()]
km_data['표준산업세세분류코드'] = km_data['표준산업세세분류코드'].astype(int).astype(str)
km_data['표준산업세세분류코드'] = km_data['표준산업세세분류코드'].apply(lambda x : x[:3])
km_data['STD_TOBZ_M_C'] = km_data['표준산업세세분류코드'].apply(lambda x : x[:2])
km_data['기준년월'] = pd.to_datetime(km_data['기준년월'], format='%Y%m')
km_data['연령대'] = km_data['연령대'].apply(exc_age)
km_data['company'] = '국민'
km_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'code_s_name', 'code_s', 'cost', 'case', 'client', 'code_m', 'company']
km_data = km_data[['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']]
km_data['case'] = km_data['case'].astype(int)

# sh data preprocessing
city_l_dic = {'전남':'전라남도', '경기':'경기도', '서울':'서울특별시'}
sex_dic = {'F':'여성', 'M':'남성'}
sh_data = pd.read_csv("dataset/신한카드.csv")
sh_data['MGPO_NM_HOM'] = sh_data['MGPO_NM_HOM'].map(city_l_dic)
sh_data['SEX_C'] = sh_data['SEX_C'].map(sex_dic)
sh_data['SL_CRI_YM'] = pd.to_datetime(sh_data['SL_CRI_YM'], format='%Y%m')
sh_data['STD_TOBZ_M_C'] = sh_data['STD_TOBZ_M_C'].astype(str)
sh_data['company'] = '신한'
sh_data = sh_data[['SL_CRI_YM', 'MGPO_NM_HOM', 'SGG_NM_HOM', 'NASD_NM_HOM', 'SEX_C', 'AGE_G5_C', 'company', 'STD_TOBZ_M_C', 'STD_TOBZ_L_C','AMT', 'CNT']]
sh_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']
sh_data['case'] = sh_data['case'].apply(lambda x : int(x) if x.isdigit() else 0)

# ss data preprocessing
def exc_age(age):
    return "60+" if int(age) >= 60 else age      
ss_data = pd.read_csv("dataset/삼성카드.csv")
ss_data['표준업종분류코드_중분류'] = ss_data['표준업종분류코드_중분류'].astype(str)
ss_data['거주지_광역시도'] = ss_data['거주지_광역시도'].map(city_l_dic)
ss_data['성별'] = ss_data['성별'].map(sex_dic)
ss_data['이용기준년월'] = pd.to_datetime(ss_data['이용기준년월'], format='%Y%m')
ss_data['연령대'] = ss_data['연령대'].apply(exc_age)
ss_data['company'] = '삼성'
ss_data = ss_data[['이용기준년월', '거주지_광역시도', '거주지_시군구', '거주지_행정동', '성별', '연령대', 'company', '표준업종분류코드_중분류', '표준업종분류코드_소분류','이용금액', '이용건수']]
ss_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']
ss_data['case'] = ss_data['case'].astype(int)

# data concat
con_df = pd.concat([km_data, sh_data, ss_data]).reset_index(drop=True) # 3사 데이터 통합
con_df = con_df.drop_duplicates()
jg_df = con_df[con_df['city_m']=='중구'].reset_index(drop=True) # 중구 data만 filtering
ddc_df = con_df[con_df['city_m']=='동두천시'].reset_index(drop=True) # 동두천시 data만 filtering
nj_df = con_df[con_df['city_m']=='나주시'].reset_index(drop=True) # 나주시 data만 filtering

In [7]:
jg_df.to_csv("jg_df.csv", index=False)
ddc_df.to_csv("ddc_df.csv", index=False)
nj_df.to_csv("nj_df.csv", index=False)

In [4]:
con_df

,date,city_l,city_m,city_s,sex,age,company,code_m,code_s,cost,case
0,2023-04-01,서울특별시,중구,다산동,여성,50,국민,56,561,24370810,683
1,2023-05-01,서울특별시,중구,약수동,남성,60+,국민,59,591,20000,3
2,2023-08-01,서울특별시,중구,신당5동,남성,60+,국민,47,478,129200,3
3,2023-08-01,경기도,동두천시,불현동,여성,30,국민,47,478,79500,7
4,2023-04-01,경기도,동두천시,생연1동,여성,20,국민,91,912,154900,17
...,...,...,...,...,...,...,...,...,...,...,...
649914,2023-01-01,서울특별시,중구,소공동,여성,30,삼성,90,902,31000,2
649915,2023-03-01,경기도,동두천시,송내동,여성,20,삼성,90,902,54000,2
649916,2023-11-01,경기도,동두천시,생연1동,남성,50,삼성,90,902,8000,2
649917,2023-03-01,서울특별시,중구,중림동,남성,40,삼성,90,902,137000,13


In [16]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32
...,...,...,...,...,...,...,...,...
1655,Vietnam,Asia,2007,74.249,85262356,2441.576404,VNM,704
1667,West Bank and Gaza,Asia,2007,73.422,4018332,3025.349798,PSE,275
1679,"Yemen, Rep.",Asia,2007,62.698,22211743,2280.769906,YEM,887
1691,Zambia,Africa,2007,42.384,11746035,1271.211593,ZMB,894


In [18]:
import plotly.express as px
# 데이터 불러오기
df = px.data.gapminder().query("year == 2007")

# 그래프 그리기
fig = px.bar(df, y="pop", x="continent", color="continent", color_discrete_sequence=px.colors.qualitative.G10)

fig.show()

In [5]:
# data concat
jg_df

,date,city_l,city_m,city_s,sex,age,company,code_m,code_s,cost,case
0,2023-04-01,서울특별시,중구,다산동,여성,50,국민,56,561,24370810,683
1,2023-05-01,서울특별시,중구,약수동,남성,60+,국민,59,591,20000,3
2,2023-08-01,서울특별시,중구,신당5동,남성,60+,국민,47,478,129200,3
6,2023-03-01,서울특별시,중구,명동,여성,40,국민,56,561,407250,27
7,2023-11-01,서울특별시,중구,다산동,남성,60+,국민,47,474,696940,8
...,...,...,...,...,...,...,...,...,...,...,...
649909,2022-12-01,서울특별시,중구,명동,남성,30,삼성,90,902,32000,3
649913,2023-07-01,서울특별시,중구,다산동,남성,40,삼성,90,902,29000,2
649914,2023-01-01,서울특별시,중구,소공동,여성,30,삼성,90,902,31000,2
649917,2023-03-01,서울특별시,중구,중림동,남성,40,삼성,90,902,137000,13


In [ ]:
# 그래프 그리기
fig = px.bar(jg_df, y="city_s", x="cost", color="city_s", color_discrete_sequence=px.colors.sequential.Agsunset)

fig.show()

In [ ]:
etc_df = pd.DataFrame(jg_df['city_s'].value_counts())
etc_df['기타여부'] = ["기타" if val / total_values < threshold else label for label, val in zip(etc_df.index, etc_df.values)]
etc_df

In [ ]:
import plotly.graph_objects as go
# 'city_s' 컬럼 값에 대한 구성 비율을 파이 차트로 시각화
fig = px.pie(etc_df, 
             names=etc_df['기타여부'], 
             values=etc_df['count'],
             title='Administration Area Composition Ratio',
             color_discrete_sequence=px.colors.cyclical.IceFire,  # 색상 설정
             custom_data=[etc_df.index],  # 추가 정보를 custom_data로 설정
             labels={'customdata': 'Count'},  # custom_data에 대한 레이블 설정
             )
# 레이블을 숫자 옆에 표시
fig.update_traces(textinfo='label+percent', pull=[0.1, 0.1, 0.1])  # pull을 사용하여 일부 조각을 분리
fig.update_layout(showlegend=False)
fig.update_layout(height=700, width=1000)

# 그림을 보여주기
fig.show()

In [ ]:
jg_df['city_s'].value_counts()

In [ ]:
import plotly.express as px

value_cnt = jg_df['city_s'].value_counts()
# jg_df는 DataFrame 객체이며, 'city_s'는 시각화하고자 하는 열입니다.
fig = px.bar(value_cnt, x=value_cnt.index, y=value_cnt.values, color=jg_df['city_s'].value_counts().values, color_continuous_scale='Cividis_r')
fig.update_layout(xaxis_title='행정동', yaxis_title='결제건수')
# 그림을 보여주기
fig.show()

In [11]:
# 'city_s' 컬럼 값에 따른 'cost'의 합 계산
cost_sum_by_city = jg_df.groupby('city_s')['cost'].sum().sort_values(ascending=False)
cost_sum_by_city

city_s
약수동     157497724638
황학동     140289956139
청구동     115371679333
다산동     112676121166
중림동     111812064895
동화동     107917564068
신당동      83364730515
신당5동     77537516775
광희동      74871313654
회현동      73736342886
명동       72046522541
소공동      64666037954
필동       44596316596
장충동      37172142758
을지로동     30264878129
Name: cost, dtype: int64

In [ ]:
fig = px.pie(jg_df['sex'].value_counts(), 
                names=jg_df['sex'].value_counts().index, 
                values=jg_df['sex'].value_counts().values,
                title='Gender Composition Ratio',
                color_discrete_sequence=px.colors.qualitative.Plotly,
                hole=0.4)
# 그림을 보여주기
fig.show()

In [12]:
# 도시별 총액의 비율을 파이 차트로 시각화
fig = px.pie(cost_sum_by_city, 
             names=cost_sum_by_city.index, 
             values=cost_sum_by_city.values,
             title='City Total Cost Ratio',  # 파이 차트 제목 설정
             color_discrete_sequence=px.colors.cyclical.IceFire,  # 색상 설정
             custom_data=[cost_sum_by_city.values],  # 추가 정보를 custom_data로 설정
             labels={'customdata': 'Total Cost'},  # custom_data에 대한 레이블 설정
             )

# 레이블을 숫자 옆에 표시
fig.update_traces(textinfo='label+percent', pull=[0.1, 0.1, 0.1])  # pull을 사용하여 일부 조각을 분리
fig.update_layout(showlegend=False)
fig.update_layout(height=700, width=1000)

# 그림을 보여주기
fig.show()

In [ ]:
cost_sum_by_city = jg_df.groupby('city_s')['cost'].sum().sort_values(ascending=False)
# jg_df는 DataFrame 객체이며, 'city_s'는 시각화하고자 하는 열입니다.
fig = px.bar(cost_sum_by_city, x=cost_sum_by_city.index, y=cost_sum_by_city.values, color=cost_sum_by_city.values, color_continuous_scale='Cividis_r')
fig.update_layout(xaxis_title='행정동', yaxis_title='결제액수')
# 그림을 보여주기
fig.show()

In [5]:
import plotly.express as px
fig = px.scatter(jg_df, x="code_s", y="cost",
                 size="case", color="company", hover_name="company",
                 log_x=True, size_max=60)

In [ ]:
fig

In [7]:
jg_df['city_s'].unique()

array(['다산동', '약수동', '신당5동', '명동', '중림동', '청구동', '을지로동', '신당동', '장충동',
       '필동', '동화동', '황학동', '소공동', '광희동', '회현동'], dtype=object)

In [13]:
km_count_dic = dict(zip(km_data['code_m'].value_counts().index,km_data['code_m'].value_counts().values))
km_std = pd.DataFrame({'표준산업분류코드':sorted(km_data['code_m'].unique())})
km_std['표준산업분류(중)'] = km_std['표준산업분류코드'].map(STD_TOBZ_M_dic)
km_std['데이터건수'] = km_std['표준산업분류코드'].map(km_count_dic)
km_std.sort_values(by='표준산업분류코드').reset_index(drop=True)

,표준산업분류코드,표준산업분류(중),데이터건수
0,10,식료품 제조업,2981
1,29,기타 기계 및 장비 제조업,336
2,33,기타 제품 제조업,1977
3,35,전기 가스 증기 및 공기 조절 공급업,2459
4,41,종합 건설업,143
5,42,전문직별 공사업,60
6,45,자동차 및 부품 판매업,145
7,46,도매 및 상품 중개업,824
8,47,소매업; 자동차 제외,117038
9,49,육상 운송 및 파이프라인 운송업,17095


In [11]:
km_lst = sorted(km_std['표준산업분류코드'].unique())
sh_lst = sorted(sh_std['표준산업분류코드'].unique())
ss_lst = sorted(ss_std['표준산업분류코드'].unique())

In [12]:
std_lst = sorted(list(set(km_std['표준산업분류코드'])|set(sh_std['표준산업분류코드'])|set(ss_std['표준산업분류코드'])))

In [13]:
code = []
km = []
sh = []
ss = []
for i in std_lst:
    code.append(i)
    if i in km_lst:
        km.append('O')
    else:
        km.append('X')
    if i in sh_lst:
        sh.append('O')
    else:
        sh.append('X')
    if i in ss_lst:
        ss.append('O')
    else:
        ss.append('X')

name = []
for i in code:
    name.append(STD_TOBZ_M_dic[i])

existent_df = pd.DataFrame({'표준산업분류코드':code, '표준산업분류(중)':name, '국민':km, '신한':sh, '삼성':ss})
existent_df

,표준산업분류코드,표준산업분류(중),국민,신한,삼성
0,10,식료품 제조업,O,X,O
1,14,의복 의복 액세서리 및 모피제품 제조업,X,X,O
2,18,인쇄 및 기록매체 복제업,X,O,O
3,21,의료용 물질 및 의약품 제조업,X,O,X
4,29,기타 기계 및 장비 제조업,O,O,O
5,30,자동차 및 트레일러 제조업,X,X,O
6,33,기타 제품 제조업,O,O,O
7,34,산업용 기계 및 장비 수리업,X,O,O
8,35,전기 가스 증기 및 공기 조절 공급업,O,X,X
9,41,종합 건설업,O,X,X


In [40]:
existent_df[(existent_df['국민'] == 'O') & (existent_df['신한'] == 'O') & (existent_df['삼성'] == 'O')].to_excel('카드사_표준산업분류_공통항목.xlsx', index=False)

In [36]:
existent_df.to_excel('표준산업분류코드_존재여부2.xlsx', index=False)